In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import lxml
import pandas as pd

In [ ]:
PATH = 'chromedriver'
browser = webdriver.Chrome(PATH)

In [ ]:
cityList = ['Delhi','Mumbai','Bangalore']
specialityList = ['Dentist','Gynecologist','Pediatrician','Orthopedist','Dermatologist',
                  'Physiotherapist','Infertility%20Specialist','ophthalmologist%2F%20eye%20surgeon','Urologist',
                  'Psychiatrist','Cardiologist','dietitian%2Fnutritionist','Gastroenterologist','Neurologist',
                  'Neurosurgeon','Pulmonologist','Bariatric%20Surgeon','Rheumatologist','Chiropractor']

In [ ]:
df = pd.DataFrame(columns=['Name', 'Speciality', 'Degree', 'Experience', 'Locality', 'City', 'Score', 'NumberOfReviews', 'NumberOfStories', 'Fee'])

In [ ]:
def openSearch(city, speciality):
    link = r'https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22'+speciality+r'%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city='+city
    browser.get(link)

In [ ]:
def scrollDown():
    initialLength = browser.execute_script("return document.body.scrollHeight")
    while True:
        browserHeight = int(browser.execute_script("return document.body.scrollHeight"))
        y = browserHeight - 1400
        script = "window.scrollTo(0,"+str(y)+");"
        browser.execute_script(script)
        time.sleep(8)

        finalLength = browser.execute_script("return document.body.scrollHeight")
        if finalLength==initialLength:
            break
        initialLength = finalLength

In [ ]:
def getProfile(card):
    try:
        link = card.find_element(By.XPATH, ".//div[@class = 'info-section']/a[1]").get_attribute('href')
    except:
        return '',0
    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[-1])
    browser.get(link)
    try:
        Degree = browser.find_element(By.XPATH, ".//p[@data-qa-id='doctor-qualifications']").text
    except:
        Degree = ''
    try:
        NumberOfReview = browser.find_element(By.XPATH, ".//p[@data-qa-id='doctor-patient-experience-score']/span[2]").text.split()[0][1:]
    except:
        NumberOfReview = 0

    browser.close()
    browser.switch_to.window(browser.window_handles[0])
    return Degree,NumberOfReview

In [ ]:
def getData():
    cards = browser.find_elements(By.XPATH, "//div[@class='listing-doctor-card']")
    for card in cards:
        try:
            name = card.find_element(By.XPATH, ".//h2[@data-qa-id='doctor_name']").text
        except:
            name = ''
        print(name)
        try:
            Speciality = card.find_element(By.XPATH, ".//div[@class='u-d-flex']/span").text
        except:
            Speciality = ''
        print(Speciality)
        Degree,NumberOfReview = getProfile(card)
        print(Degree)
        try:
            Experience = card.find_element(By.XPATH, ".//div[@data-qa-id='doctor_experience']/div").text.split()[0]
        except:
            Experience = 0
        print(Experience)
        try:
            Locality = card.find_element(By.XPATH, ".//span[@data-qa-id='practice_locality']").text[:-1]
        except:
            Locality = ''
        print(Locality)
        try:
            City = card.find_element(By.XPATH, ".//span[@data-qa-id='practice_city']").text
        except:
            City = ''
        print(City)
        try:
            Score = card.find_element(By.XPATH, ".//span[@data-qa-id='doctor_recommendation']").text[:-1]
        except:
            Score = 0
        print(Score)
        print(NumberOfReview)
        try:
            NumberOfStories = card.find_element(By.XPATH, ".//span[@data-qa-id='total_feedback']").text.split()[0]
        except:
            NumberOfStories = 0
        print(NumberOfStories)
        try:
            Fee = card.find_element(By.XPATH, ".//span[@data-qa-id='consultation_fee']").text[1:]
        except:
            Fee = 0
        print(Fee)
        global df
        df = df.append({'Name' : name, 
                        'Speciality' : Speciality, 
                        'Degree' : Degree,
                        'Experience' : Experience,
                        'Locality' : Locality,
                        'City' : City,
                        'Score' : Score,
                        'NumberOfReviews' : NumberOfReview,
                        'NumberOfStories' : NumberOfStories,
                        'Fee' : Fee},
        ignore_index = True)

In [ ]:
for city in cityList:
    for speciality in specialityList:
        openSearch(city,speciality)
        try:
            totalDoctors = int(browser.find_element(By.XPATH, "//h1[@class='u-xx-large-font u-bold']").text.split()[0])
            print(totalDoctors)
        except:
            continue
        scrollDown()
        getData()
        df.to_csv(r'D:\DoctorsData.csv',index=False)

time.sleep(10)